In [1]:
import numpy as np

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import pyrotd

In [3]:
THs = pd.read_csv('motion_suit.csv', header=None)

In [4]:
def read_peer_record(file):
    with open(file) as f:
        f.readline()
        f.readline()
        f.readline()
        dt = np.float(f.readline().split(',')[1].split('=')[1][:9])
        data = []
        for line in f:
            for val in line.strip().split():
                data.append(float(val))
        return dt , np.array(data)

In [5]:
template = open('NewBirectionalAnalysis/template.inp','r').readlines()

In [6]:
template_str = ''
for line in template:
    template_str = template_str + line

In [7]:
time_histories = {}
for i,th in THs.iterrows():
    dt, data = read_peer_record('PEERNGARecords_Unscaled'+ '/' + th[0].split('\\')[-1])
    time_histories[i] = {'file':th[0].split('\\')[-1], 'scale':th[1], 'time_history':data, 'dt':dt}
    print(th[0].split('\\')[-1],1/dt)

RSN164_IMPVALL.H_H-CPE147.AT2 100.0
RSN286_ITALY_A-BIS000.AT2 344.82758620689657
RSN289_ITALY_A-CTR000.AT2 416.6666666666667
RSN313_CORINTH_COR--L.AT2 100.0
RSN755_LOMAP_CYC195.AT2 200.0
RSN827_CAPEMEND_FOR000.AT2 50.0
RSN864_LANDERS_JOS000.AT2 50.0
RSN1083_NORTHR_GLE170.AT2 100.0
RSN1614_DUZCE_1061-N.AT2 100.0
RSN1616_DUZCE_362-N.AT2 100.0
RSN1633_MANJIL_ABBAR--L.AT2 50.0
RSN3750_CAPEMEND_LFS270.AT2 200.0
RSN3757_LANDERS_NPF090.AT2 200.0
RSN4013_SANSIMEO_36258021.AT2 200.0
RSN4841_CHUETSU_65004NS.AT2 100.0
RSN4843_CHUETSU_65006NS.AT2 100.0
RSN4844_CHUETSU_65007NS.AT2 100.0
RSN4848_CHUETSU_65011NS.AT2 100.0
RSN4850_CHUETSU_65013NS.AT2 100.0
RSN4872_CHUETSU_65053NS.AT2 100.0
RSN4882_CHUETSU_65321NS.AT2 100.0
RSN5270_CHUETSU_NIG024NS.AT2 100.0
RSN5274_CHUETSU_NIG028NS.AT2 100.0
RSN5284_CHUETSU_NIGH11NS.AT2 200.0
RSN5775_IWATE_54009NS.AT2 100.0
RSN5776_IWATE_54010NS.AT2 100.0
RSN5783_IWATE_54026NS.AT2 100.0
RSN5800_IWATE_55441NS.AT2 100.0
RSN5806_IWATE_55461NS.AT2 100.0
RSN5818_IWATE_48A6

In [8]:
PAIR = np.zeros([30,2], dtype=int)
PAIR[:,0] = np.arange(0,30)
PAIR[:,1] = np.arange(29,-1,-1)

In [9]:
coda_duration = 10 #second
cpu = 10
batfile = 'NewBirectionalAnalysis/run.bat'
subm_fmt = 'abq2023 input={0:s}.inp job={0:s} cpus={1:d} mp_mode=threads\n'
bat_file = open(batfile,'w')
for pp in PAIR:
    file1 = time_histories[pp[0]]['file']
    file2 = time_histories[pp[1]]['file']
    inp_fname = 'NewBirectionalAnalysis/{0:s}_{1:s}.inp'.format(\
            file1.split('_')[0],file2.split('_')[0])
    bat_file.write(subm_fmt.format(inp_fname.split('/')[-1][:-4],cpu))
    with open(inp_fname,'w') as f:
        th_str1 = ''
        for j,dd in enumerate(time_histories[pp[0]]['time_history']):
            if (j%4 == 0):
                th_str1 = th_str1 + '\n{0:15.8f},{1:15.8f}'.format(j*time_histories[pp[0]]['dt'],dd*time_histories[pp[0]]['scale'])
            else:
                th_str1 = th_str1 + ',{0:15.8f},{1:15.8f}'.format(j*time_histories[pp[0]]['dt'],dd*time_histories[pp[0]]['scale'])
        th_str2 = ''
        for j,dd in enumerate(time_histories[pp[0]]['time_history']):
            if (j%4 == 0):
                th_str2 = th_str2 + '\n{0:15.8f},{1:15.8f}'.format(j*time_histories[pp[1]]['dt'],dd*time_histories[pp[1]]['scale'])
            else:
                th_str2 = th_str2 + ',{0:15.8f},{1:15.8f}'.format(j*time_histories[pp[1]]['dt'],dd*time_histories[pp[1]]['scale'])
        DT = min(time_histories[pp[0]]['dt'],time_histories[pp[1]]['dt'])
        DUR = max(time_histories[pp[0]]['dt']*len(time_histories[pp[1]]['time_history']),\
                  time_histories[pp[1]]['dt']*len(time_histories[pp[1]]['time_history']))
        f.write(template_str.format(file1.split('_')[0],th_str1[1:-1],\
                file2.split('_')[0],th_str2[1:-1],\
                DT,DUR+coda_duration,\
                file1.split('_')[0], file2.split('_')[0]))
bat_file.close()

TypeError: write() takes exactly one argument (8 given)

In [ ]:
time_histories[pp[0]]